# Spliting Data

This notebook focuses on exploration using two essential files: the annotations data extracted from the actual screening profile (available in the [IDR repository](https://github.com/IDR/idr0133-dahlin-cellpainting/tree/main/screenA)) and the metadata retrieved from the supplementary section of the [research paper](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-023-36829-x/MediaObjects/41467_2023_36829_MOESM5_ESM.xlsx).

We explore the number of unique compounds associated with each cell injury and subsequently cross-reference this information with the screening profile. The aim is to assess the feasibility of using the data for training a machine learning model to predict cell injury.

Next, with the information collected, we then split our data, identify the controls, and determine our training, test, and holdout sets for subsequent analysis.


In [20]:
import json
import pathlib
from collections import defaultdict

import pandas as pd
import numpy as np

## Paramters

Below are the parameters defined that are used in this notebook

----

In [21]:
# data directory
data_dir = pathlib.Path("../../data").resolve(strict=True)
results_dir = pathlib.Path("../../results").resolve(strict=True)
data_split_dir = (results_dir / "0.data_splits").resolve()
data_split_dir.mkdir(exist_ok=True)

# data paths
suppl_meta_path = data_dir / "41467_2023_36829_MOESM5_ESM.csv.gz"
screen_anno_path = data_dir / "idr0133-screenA-annotation.csv.gz"

# load data
image_profile_df = pd.read_csv(screen_anno_path)
meta_df = image_profile_df[image_profile_df.columns[:31]]
compounds_df = meta_df[["Compound Name", "Compound Class"]]

suppl_meta_df = pd.read_csv(suppl_meta_path)
cell_injury_df = suppl_meta_df[["Cellular injury category", "Compound alias"]]

## Data splitting and exploration 

In this notebook, we explore the structure of the experiments, focusing on how treatments were applied, conducting quality data checks, and identifying other relationships within the dataset.

Additionally, we extract datasets that will be utilized in our machine learning processes. These datasets encompass our training data, control sets, and holdout sets.

-----

The code below, we extract information regarding various injury types and the corresponding number of compounds known to induce each type of injury.
then, we cross-reference with the selected compounds and identify wells that have a match in the profile data.

This will be our training dataset, where it contains all the cellular injury labeles and it will be used for machine learning modeling 

In [22]:
# getting profilies based on injury and compound type
injury_and_compounds = defaultdict(list)
for injury, compound in cell_injury_df.values.tolist():
    injury_and_compounds[injury].append(compound)

# cross reference selected injury and associated components into the screen profile
injury_profiles = []
for injury_type, compound_list in injury_and_compounds.items():
    sel_profile = image_profile_df[
        image_profile_df["Compound Name"].isin(compound_list)
    ]
    sel_profile.insert(0, "injury_type", injury_type)
    injury_profiles.append(sel_profile)

In [23]:
# creating a dataframe that contains stratified screen Data
injured_df = pd.concat(injury_profiles)

# display df
print(injured_df.shape)
injured_df.head()

(9576, 404)


injury_type       Plate Well Characteristics [Organism]  \
158  Cytoskeletal  BR00110363  E17               Homo sapiens   
159  Cytoskeletal  BR00110363  E18               Homo sapiens   
160  Cytoskeletal  BR00110363  E19               Homo sapiens   
161  Cytoskeletal  BR00110363  E20               Homo sapiens   
162  Cytoskeletal  BR00110363  E21               Homo sapiens   

    Term Source 1 REF Term Source 1 Accession Characteristics [Cell Line]  \
158         NCBITaxon          NCBITaxon_9606                        U2OS   
159         NCBITaxon          NCBITaxon_9606                        U2OS   
160         NCBITaxon          NCBITaxon_9606                        U2OS   
161         NCBITaxon          NCBITaxon_9606                        U2OS   
162         NCBITaxon          NCBITaxon_9606                        U2OS   

    Term Source 2 REF Term Source 2 Accession  \
158               EFO             EFO_0002869   
159               EFO             EFO_0002869   
160               EFO             EFO_0002869   
161               EFO             EFO_0002869   
162               EFO             EFO_0002869   

     Experimental Condition [Treatment time (h)]  ...  \
158                                           24  ...   
159                                           24  ...   
160                                           24  ...   
161                                           24  ...   
162                                           24  ...   

     Nuclei_Texture_InverseDifferenceMoment_DNA_5_0  \
158                                        0.561075   
159                                        0.642707   
160                                        0.599857   
161                                        0.513671   
162                                        0.402869   

     Nuclei_Texture_InverseDifferenceMoment_RNA_5_0  \
158                                        0.139535   
159                                        0.052501   
160                                        0.184587   
161                                        0.137843   
162                                        0.083364   

    Nuclei_Texture_SumAverage_AGP_5_0 Nuclei_Texture_SumAverage_DNA_10_0  \
158                          0.188096                          -1.035562   
159                          0.130166                          -1.304556   
160                          0.111444                          -1.462714   
161                          0.165498                          -1.005157   
162                          0.181626                          -1.068167   

    Nuclei_Texture_SumAverage_Mito_5_0 Nuclei_Texture_SumAverage_RNA_5_0  \
158                           0.655389                          0.182888   
159                           0.438742                          0.187985   
160                           0.821791                          0.229490   
161                           0.264772                          0.169579   
162                           0.469826                          0.411077   

     Nuclei_Texture_SumEntropy_DNA_10_0  Nuclei_Texture_SumEntropy_DNA_20_0  \
158                           -0.004066                            0.130472   
159                            0.088121                            0.289709   
160                            0.121207                            0.165713   
161                            0.142331                            0.264883   
162                            0.427186                            0.458690   

    Nuclei_Texture_SumEntropy_DNA_5_0 Nuclei_Texture_SumVariance_DNA_20_0  
158                         -0.418286                            0.283484  
159                         -0.451626                            0.461128  
160                         -0.342221                            0.388047  
161                         -0.161366                            0.337277  
162                         -0.012347                            0.658387  

[5 row

>This DataFrame categorizes wells based on their injury types and with its corresponding compounds linked to each specific injury type.
> Note the new column `injury_type` indicating the assigned injury type for each well.
> This assignment is determined by the component with which the well has been treated.

In [24]:
neg_control_df = image_profile_df.loc[image_profile_df["Compound Name"] == "DMSO"]
neg_control_df.insert(0, "injury_type", "Control")
neg_control_df.to_csv("negative_control.csv.gz", compression="gzip", index=False)

print("Shape:", neg_control_df.shape)
neg_control_df.head()

Shape: (9855, 404)


injury_type       Plate Well Characteristics [Organism] Term Source 1 REF  \
0     Control  BR00110363   B2               Homo sapiens         NCBITaxon   
1     Control  BR00110363   B3               Homo sapiens         NCBITaxon   
2     Control  BR00110363   B4               Homo sapiens         NCBITaxon   
3     Control  BR00110363   B5               Homo sapiens         NCBITaxon   
4     Control  BR00110363   B6               Homo sapiens         NCBITaxon   

  Term Source 1 Accession Characteristics [Cell Line] Term Source 2 REF  \
0          NCBITaxon_9606                        U2OS               EFO   
1          NCBITaxon_9606                        U2OS               EFO   
2          NCBITaxon_9606                        U2OS               EFO   
3          NCBITaxon_9606                        U2OS               EFO   
4          NCBITaxon_9606                        U2OS               EFO   

  Term Source 2 Accession  Experimental Condition [Treatment time (h)]  ...  \
0             EFO_0002869                                           24  ...   
1             EFO_0002869                                           24  ...   
2             EFO_0002869                                           24  ...   
3             EFO_0002869                                           24  ...   
4             EFO_0002869                                           24  ...   

   Nuclei_Texture_InverseDifferenceMoment_DNA_5_0  \
0                                        0.000098   
1                                        0.025857   
2                                        0.041060   
3                                        0.022156   
4                                        0.007213   

   Nuclei_Texture_InverseDifferenceMoment_RNA_5_0  \
0                                        0.057244   
1                                        0.099848   
2                                        0.119247   
3                                        0.036473   
4                                        0.023068   

  Nuclei_Texture_SumAverage_AGP_5_0 Nuclei_Texture_SumAverage_DNA_10_0  \
0                          0.160847                          -0.083034   
1                          0.017477                           0.021300   
2                          0.111741                           0.041592   
3                         -0.013141                           0.008690   
4                          0.110361                           0.054405   

  Nuclei_Texture_SumAverage_Mito_5_0 Nuclei_Texture_SumAverage_RNA_5_0  \
0                          -0.023290                         -0.066369   
1                           0.058137                         -0.097280   
2                           0.224199                         -0.088845   
3                           0.060860                          0.044924   
4                           0.030157                          0.066480   

   Nuclei_Texture_SumEntropy_DNA_10_0  Nuclei_Texture_SumEntropy_DNA_20_0  \
0                           -0.015235                           -0.035909   
1                           -0.073545                           -0.044883   
2                            0.000327                           -0.003115   
3                            0.040528                            0.070877   
4                            0.038910                            0.048559   

  Nuclei_Texture_SumEntropy_DNA_5_0 Nuclei_Texture_SumVariance_DNA_20_0  
0                         -0.013321                           -0.032067  
1                         -0.089842                           -0.015240  
2                          0.016075                           -0.014406  
3                          0.038779                            0.072871  
4                          0.050371                            0.056829  

[5 rows x 404 columns]

> This dataframe contains all the wells that are considered to be controls. 
> All of these wells have been treated with DMSO

Now that our dataset is labeled, the next step is to concatenate them and begin splitting the dataset into training, testing, and holdout sets. Additionally, we'll extract metadata from the labeled data to gain a better understanding of how the treatments and plates were generated in this experiment.

In [25]:
# creating labeled_df
labeled_df = pd.concat([neg_control_df, injured_df])

# save and display
print("shape:", labeled_df.shape)
labeled_df.head()

shape: (19431, 404)


injury_type       Plate Well Characteristics [Organism] Term Source 1 REF  \
0     Control  BR00110363   B2               Homo sapiens         NCBITaxon   
1     Control  BR00110363   B3               Homo sapiens         NCBITaxon   
2     Control  BR00110363   B4               Homo sapiens         NCBITaxon   
3     Control  BR00110363   B5               Homo sapiens         NCBITaxon   
4     Control  BR00110363   B6               Homo sapiens         NCBITaxon   

  Term Source 1 Accession Characteristics [Cell Line] Term Source 2 REF  \
0          NCBITaxon_9606                        U2OS               EFO   
1          NCBITaxon_9606                        U2OS               EFO   
2          NCBITaxon_9606                        U2OS               EFO   
3          NCBITaxon_9606                        U2OS               EFO   
4          NCBITaxon_9606                        U2OS               EFO   

  Term Source 2 Accession  Experimental Condition [Treatment time (h)]  ...  \
0             EFO_0002869                                           24  ...   
1             EFO_0002869                                           24  ...   
2             EFO_0002869                                           24  ...   
3             EFO_0002869                                           24  ...   
4             EFO_0002869                                           24  ...   

   Nuclei_Texture_InverseDifferenceMoment_DNA_5_0  \
0                                        0.000098   
1                                        0.025857   
2                                        0.041060   
3                                        0.022156   
4                                        0.007213   

   Nuclei_Texture_InverseDifferenceMoment_RNA_5_0  \
0                                        0.057244   
1                                        0.099848   
2                                        0.119247   
3                                        0.036473   
4                                        0.023068   

  Nuclei_Texture_SumAverage_AGP_5_0 Nuclei_Texture_SumAverage_DNA_10_0  \
0                          0.160847                          -0.083034   
1                          0.017477                           0.021300   
2                          0.111741                           0.041592   
3                         -0.013141                           0.008690   
4                          0.110361                           0.054405   

  Nuclei_Texture_SumAverage_Mito_5_0 Nuclei_Texture_SumAverage_RNA_5_0  \
0                          -0.023290                         -0.066369   
1                           0.058137                         -0.097280   
2                           0.224199                         -0.088845   
3                           0.060860                          0.044924   
4                           0.030157                          0.066480   

   Nuclei_Texture_SumEntropy_DNA_10_0  Nuclei_Texture_SumEntropy_DNA_20_0  \
0                           -0.015235                           -0.035909   
1                           -0.073545                           -0.044883   
2                            0.000327                           -0.003115   
3                            0.040528                            0.070877   
4                            0.038910                            0.048559   

  Nuclei_Texture_SumEntropy_DNA_5_0 Nuclei_Texture_SumVariance_DNA_20_0  
0                         -0.013321                           -0.032067  
1                         -0.089842                           -0.015240  
2                          0.016075                           -0.014406  
3                          0.038779                            0.072871  
4                          0.050371                            0.056829  

[5 rows x 404 columns]

Here, we're storing the metadata and feature column names into a JSON file to simplify loading during feature engineering processes.

This will be saved in the `results/0.data_splits` directory

In [26]:
# collecting metadata and feature column names
feature_cols = labeled_df.columns[32:].tolist()
raw_features = {
    "compartments": list(set([name.split("_")[0] for name in feature_cols])),
    "meta_features": injured_df.columns[:32].tolist(),
    "feature_cols": feature_cols,
}

# saving into JSON file
with open(data_split_dir / "raw_feature_names.json", mode="w") as stream:
    json.dump(raw_features, stream)

Next we wanted to extract some metadata regarding how many compound and wells are treated with a given compounds

This will be saved in the `results/0.data_splits` directory

In [27]:
meta_injury = []
for injury_type, df in labeled_df.groupby("injury_type"):
    # extract n_wells, n_compounds and unique compounds per injury_type
    n_wells = df.shape[0]
    unique_compounds = list(df["Compound Name"].unique())
    n_compounds = len(unique_compounds)

    # store information
    meta_injury.append([injury_type, n_wells, n_compounds, unique_compounds])

injury_meta_df = pd.DataFrame(
    meta_injury, columns=["injury_type", "n_wells", "n_compounds", "compound_list"]
).sort_values("n_wells", ascending=False)
injury_meta_df.to_csv(data_split_dir / "injury_well_counts_table.csv", index=False)

# display
print("shape:", injury_meta_df.shape)
injury_meta_df

shape: (15, 4)


injury_type  n_wells  n_compounds  \
0                Control     9855            1   
1           Cytoskeletal     1472           15   
7          Miscellaneous     1304           39   
6                 Kinase     1104           13   
3              Genotoxin      944           22   
5                  Hsp90      552            3   
11                 Redox      312           12   
12               Saponin      288           11   
4                   HDAC      168            5   
8           Mitochondria      144            4   
10            Proteasome      144            4   
9   Nonspecific reactive      128            6   
2            Ferroptosis       96            4   
13                Tannin       96            4   
14                  mTOR       96            2   

                                        compound_list  
0                                              [DMSO]  
1   [Nocodazole, Colchicine, Paclitaxel, Vinblasti...  
7   [L-Buthionine-(S,R)-sulfoximine, CDDO Im, Cino...  
6   [Wortmannin, Staurosporine, PI-103, BEZ-235, A...  
3   [Camptothecin, CX-5461, Doxorubicin, Cladribin...  
5                   [Radicicol, Geldanamycin, 17-AAG]  
11  [Menadione, PKF118-310, 4-Amino-1-naphthol (HC...  
12  [Digitonin, Saikosaponin A, Polygalasaponin F,...  
4     [AR-42, SAHA, ITF 2357, Panobinostat, Apicidin]  
8         [Antimycin A, CCCP, Rotenone, Oligomycin A]  
10    [Carfilzomib, Bortezomib, (S)-MG132, (R)-MG132]  
9   [nan, Ebselen, IPA-3, 2-Tert-butyl-1,4-benzoqu...  
2             [ML-162, ML-210, (1S,3R)-RSL3, Erastin]  
13  [Gallotannin, Corilagin, Chebulagic acid, Puni...  
14                               [Torin 1, Rapamycin]

> This DataFrame contains information about wells associated with a specific injury type.
> It includes details such as the number of components used along with the list of the components responsible for the identified injury type.

Next, we construct the profile metadata. This provides a structured overview of how the treatments assicoated with injuries were applied, detailing the treatments administered to each plate.

This will be saved in the `results/0.data_splits` directory

In [28]:
injury_meta_dict = {}
for injury, df in labeled_df.groupby("injury_type"):
    # collecting treatment metadata
    plates = df["Plate"].unique().tolist()
    treatment_meta = {}
    treatment_meta["n_plates"] = len(plates)
    treatment_meta["n_wells"] = df.shape[0]
    treatment_meta["n_treatments"] = len(df["Compound Name"].unique())
    treatment_meta["associated_plates"] = plates

    # counting treatments
    treatment_counter = {}
    for treatment, df2 in df.groupby("Compound Name"):
        if treatment is np.nan:
            continue
        n_treatments = df2.shape[0]
        treatment_counter[treatment] = n_treatments

    # storing treatment counts
    treatment_meta["treatments"] = treatment_counter
    injury_meta_dict[injury] = treatment_meta

# save dictionary into a json file
with open(data_split_dir / "injury_metadata.json", mode="w") as stream:
    json.dump(injury_meta_dict, stream)

Here we build a plate metadata infromations where we look at the type of treatments and amount of wells with the treatment that are present in the dataset

This will be saved in `results/0.data_splits`

In [29]:
plate_meta = {}
for plate_id, df in labeled_df.groupby("Plate"):
    unique_compounds = list(df["Compound Name"].unique())
    n_treatments = len(unique_compounds)

    # counting treatments
    treatment_counter = {}
    for treatment, df2 in df.groupby("Compound Name"):
        n_treatments = df2.shape[0]
        treatment_counter[treatment] = n_treatments

    plate_meta[plate_id] = treatment_counter

# save dictionary into a json file
with open(data_split_dir / "plate_info.json", mode="w") as stream:
    json.dump(plate_meta, stream)

## Data Splitting 
---

### Holdout Dataset

Here we collected out holdout dataset. The holdout dataset is a subset of the dataset that is not used during model training or tuning. Instead, it is reserved solely for evaluating the model's performance after it has been trained.

In this notebook, we will include three different types of held-out datasets before proceeding with our machine learning training and evaluation.



In [30]:
#### Plate heldout dataset

### Training and Test sets 